# Linear Models: OLS

In [130]:
import pandas as pd
import numpy as np

In [131]:
clean = pd.read_csv("../data/clean.csv")
clean.head()

,crop_count,crop_name_1,crop_harvest_1,crop_intercrop_1,ID_PROJ,ID_COUNTRY,YEAR,SURVEY_ID,ID_HH,Country,...,value_livestock_production_USD_PPP_pHH_Yr,value_livestock_prod_consumed_USD_PPP_pHH_Yr,Market_Orientation,Livestock_Orientation,Food_Availability_kCal_MAE_day,Food_Self_Sufficiency_kCal_MAE_day,NrofMonthsWildFoodCons,Gender_MaleControl,Gender_FemaleControl,continent
0,NaN,groundnut,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_1_1,Tanzania,...,38.500640,0.000000,1.000000,0.049505,3650.565755,0.000000,0,0.000000,1.000000,africa
1,NaN,millet,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_2_1,Tanzania,...,0.000000,0.000000,1.000000,0.000000,3632.448997,0.000000,4,0.499999,0.500001,africa
2,NaN,groundnut,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_3_1,Tanzania,...,128.335467,0.000000,1.000000,0.129870,5314.779021,1750.000000,1,1.000000,0.000000,africa
3,NaN,cowpea,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_4_1,Tanzania,...,0.000000,0.000000,NaN,NaN,257.234727,257.234727,0,NaN,NaN,africa
4,NaN,sesame,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_5_1,Tanzania,...,233.994961,23.102694,0.931377,0.695041,925.310354,22.070240,0,1.000000,0.000000,africa


In [132]:
from Functions import Cleaning_Functions
fun = Cleaning_Functions()

In [133]:
#clean.loc[:, "Market_Orientation"]
clean.head()
#nums = clean.select_dtypes(include='number')

,crop_count,crop_name_1,crop_harvest_1,crop_intercrop_1,ID_PROJ,ID_COUNTRY,YEAR,SURVEY_ID,ID_HH,Country,...,value_livestock_production_USD_PPP_pHH_Yr,value_livestock_prod_consumed_USD_PPP_pHH_Yr,Market_Orientation,Livestock_Orientation,Food_Availability_kCal_MAE_day,Food_Self_Sufficiency_kCal_MAE_day,NrofMonthsWildFoodCons,Gender_MaleControl,Gender_FemaleControl,continent
0,NaN,groundnut,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_1_1,Tanzania,...,38.500640,0.000000,1.000000,0.049505,3650.565755,0.000000,0,0.000000,1.000000,africa
1,NaN,millet,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_2_1,Tanzania,...,0.000000,0.000000,1.000000,0.000000,3632.448997,0.000000,4,0.499999,0.500001,africa
2,NaN,groundnut,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_3_1,Tanzania,...,128.335467,0.000000,1.000000,0.129870,5314.779021,1750.000000,1,1.000000,0.000000,africa
3,NaN,cowpea,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_4_1,Tanzania,...,0.000000,0.000000,NaN,NaN,257.234727,257.234727,0,NaN,NaN,africa
4,NaN,sesame,NaN,NaN,CFM,TZ,2015,TZ_CFM_2015,TZ_2015_CFM_5_1,Tanzania,...,233.994961,23.102694,0.931377,0.695041,925.310354,22.070240,0,1.000000,0.000000,africa


In [134]:
def standarize_data(df):
        """
        Input: a dataset
        action: returns numeric column values scaled by mean and standard deviation
        """
        numeric_data = df.select_dtypes(include=['float64', 'int64'])
        for i in numeric_data.columns:
            df[i] = (df[i] - df[i].mean())/df[i].std()
        return df

In [135]:
#for market orientash
#Clean up market orientation data
market_data = fun.delete_id_columns(clean) #1
print("1")
market_data, pred_market = fun.drop_response_rows_with_NAs(market_data, "Market_Orientation", "PPI_Likelihood") #2
print("2")
market_data = fun.replace_NAN_with_na(market_data) #3
print("3")
market_data = fun.entry_to_lowercase(market_data) #4
print("4")
market_data = fun.remove_underscores_spaces(market_data) #5
print("5")
market_data = fun.convert_to_categorical(market_data) #6
print("6")
market_data = fun.impute_data(market_data)
print("7")
market_data = standarize_data(market_data)
market_data.head()

1
2
3
4
5
6
7


,crop_count,crop_name_1,crop_harvest_1,crop_intercrop_1,YEAR,Country,HHsizemembers,HHsizeMAE,HouseholdType,Head_EducationLevel,...,value_livestock_production_USD_PPP_pHH_Yr,value_livestock_prod_consumed_USD_PPP_pHH_Yr,Market_Orientation,Livestock_Orientation,Food_Availability_kCal_MAE_day,Food_Self_Sufficiency_kCal_MAE_day,NrofMonthsWildFoodCons,Gender_MaleControl,Gender_FemaleControl,continent
0,-0.207464,groundnut,na,na,-2.014476,tanzania,-0.498593,-0.456316,single,primary,...,-0.021393,-0.013133,0.498261,-0.913541,-0.014830,-0.014185,-0.593084,-2.384182,2.384182,africa
1,-0.207464,millet,na,na,-2.014476,tanzania,0.273052,-0.037211,together,primary,...,-0.021823,-0.013133,0.498261,-1.026920,-0.014830,-0.014185,0.836612,-0.801700,0.801700,africa
2,-0.207464,groundnut,na,na,-2.014476,tanzania,-0.369985,-0.290662,together,illiterate,...,-0.020390,-0.013133,0.498261,-0.729482,-0.014826,-0.014181,-0.235660,0.780789,-0.780789,africa
3,-0.207464,sesame,na,na,-2.014476,tanzania,-0.241378,-0.110099,together,secondary,...,-0.019210,-0.012850,0.277792,0.564913,-0.014836,-0.014185,-0.593084,0.780789,-0.780789,africa
4,-0.207464,groundnut,na,na,-2.014476,tanzania,-0.112770,-0.168078,together,primary,...,-0.019888,-0.011010,-2.714514,1.263351,-0.014837,-0.014184,1.194036,0.780789,-0.780789,africa


In [136]:
#code by ana
def get_dummyXs_y(df, y_var):
    
    y = df[y_var]
    X  = df.drop(y_var, axis = 1)
    X_cat = X.select_dtypes(include = ["category", "O"])
    X_num = X.select_dtypes(include=['float64', 'int64'])
    
    X_cat_dummy = pd.get_dummies(X_cat)
    newX = pd.concat([X_num, X_cat_dummy], axis = 1)
    
    return newX, y

In [137]:
market_nums = market_data.select_dtypes(include='number')
X_market = get_dummyXs_y(market_data, "Market_Orientation")[0]
y_market = get_dummyXs_y(market_data, "Market_Orientation")[1]

In [138]:
#X and Y for market orientation
from sklearn.model_selection import train_test_split

X = market_nums.loc[:, market_nums.columns != 'Market_Orientation']
y = market_nums.loc[:, "Market_Orientation"]
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size = .3, random_state = 2021
                                            )

In [139]:
#1. Fit OLS on the training dataset and compute the test error. Just numerics
from sklearn import model_selection, linear_model, neighbors, preprocessing, metrics 
from sklearn.preprocessing import Normalizer
scaler = preprocessing.StandardScaler()
#print(X_tr)
#Xtrain = scaler.fit_transform(Xtrain)
#Xtest = scaler.transform(Xtest)

reg = linear_model.LinearRegression()
reg.fit(Xtrain, ytrain)
predict = reg.predict(Xtest)
MSE_OLS = metrics.mean_squared_error(ytest,predict)
print(MSE_OLS)

0.7940168579991338


In [140]:
import statsmodels.api as sm

mod = sm.OLS(ytrain, Xtrain)
res = mod.fit()

print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:     Market_Orientation   R-squared (uncentered):                   0.205
Model:                            OLS   Adj. R-squared (uncentered):              0.202
Method:                 Least Squares   F-statistic:                              65.70
Date:                Tue, 08 Jun 2021   Prob (F-statistic):                   2.24e-305
Time:                        17:10:10   Log-Likelihood:                         -8685.1
No. Observations:                6645   AIC:                                  1.742e+04
Df Residuals:                    6619   BIC:                                  1.760e+04
Df Model:                          26                                                  
Covariance Type:            nonrobust                                                  
                                                   coef    std err          t      P>|t|      [0.025      0.975]
-------

In [141]:
#X and Y for market orientation ALL with dummy
from sklearn.model_selection import train_test_split

X = market_nums.loc[:, market_nums.columns != 'Market_Orientation']
y = market_nums.loc[:, "Market_Orientation"]
Xtrain, Xtest, ytrain, ytest = train_test_split(X_market,y_market,test_size = .3, random_state = 50)
#1. Fit OLS on the training dataset and compute the test error. ALL with dummy
from sklearn import model_selection, linear_model, neighbors, preprocessing, metrics 
from sklearn.preprocessing import Normalizer
scaler = preprocessing.StandardScaler()
#print(X_tr)
#Xtrain = scaler.fit_transform(Xtrain)
#Xtest = scaler.transform(Xtest)

reg = linear_model.LinearRegression()
reg.fit(Xtrain, ytrain)
predict = reg.predict(Xtest)
MSE_OLS = metrics.mean_squared_error(ytest,predict)
print(MSE_OLS)

2.7761319829065335e+19


In [142]:
import statsmodels.api as sm

mod = sm.OLS(ytrain, Xtrain)
res = mod.fit()

print(res.summary())

                            OLS Regression Results                            
Dep. Variable:     Market_Orientation   R-squared:                       0.354
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                     26.85
Date:                Tue, 08 Jun 2021   Prob (F-statistic):               0.00
Time:                        17:10:12   Log-Likelihood:                -7981.4
No. Observations:                6645   AIC:                         1.623e+04
Df Residuals:                    6511   BIC:                         1.714e+04
Df Model:                         133                                         
Covariance Type:            nonrobust                                         
                                                   coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

In [144]:
betas_ppi = reg.coef_
max5_index_ppi = [list(abs(betas_ppi)).index(x) for x in np.sort(abs(betas_ppi))[::-1][:20]]
min5_index_ppi = [list(abs(betas_ppi)).index(x) for x in np.sort(abs(betas_ppi))[:10]]
betas_mo[min5_index_ppi]
Xtrain.columns[min5_index_ppi]
pd.DataFrame([Xtrain.columns[min5_index_ppi], betas_ppi[min5_index_ppi]]).T

,0,1
0,score_HDDS_purchasedBadSeason,0.0003696
1,NrofMonthsWildFoodCons,0.00642886
2,YEAR,0.00645827
3,TVA_USD_PPP_pmae_pday,0.00777081
4,HHsizemembers,-0.00870459
5,score_HDDS_farmbasedGoodSeason,0.00968723
6,NrofMonthsFoodInsecure,0.0116174
7,crop_count,-0.0128446
8,score_HDDS_purchasedGoodSeason,-0.016325
9,score_HDDS_GoodSeason,0.0212405


In [115]:
#for PPI Threshold
market_PPI = fun.delete_id_columns(clean) #1
print("1")
market_PPI, pred_PPI = fun.drop_response_rows_with_NAs(market_PPI, "PPI_Likelihood", "Market_Orientation") #2
print("2")
market_PPI = fun.replace_NAN_with_na(market_PPI) #3
print("3")
market_PPI = fun.entry_to_lowercase(market_PPI) #4
print("4")
market_PPI = fun.remove_underscores_spaces(market_PPI) #5
print("5")
market_PPI = fun.convert_to_categorical(market_PPI) #6
print("6")
market_PPI = fun.impute_data(market_PPI)
print("7")

1
2
3
4
5
6
7


In [116]:
PPI_nums = market_PPI.select_dtypes(include='number')
X_PPI = get_dummyXs_y(market_PPI, "PPI_Likelihood")[0]
y_PPI = get_dummyXs_y(market_PPI, "PPI_Likelihood")[1]

In [117]:
#X and Y for PPI_threshold
from sklearn.model_selection import train_test_split

X_p = PPI_nums.loc[:, PPI_nums.columns != 'PPI_Likelihood']
y_p = PPI_nums.loc[:, "PPI_Likelihood"]
Xtrain, Xtest, ytrain, ytest = train_test_split(X_p,y_p,test_size = .3, random_state = 5)

In [118]:
#1. Fit OLS on the training dataset and compute the test error. Just numerics
from sklearn import model_selection, linear_model, neighbors, preprocessing, metrics 
from sklearn.preprocessing import Normalizer
#scaler = preprocessing.StandardScaler()
#print(X_tr)
#Xtrain = scaler.fit_transform(Xtrain)
#Xtest = scaler.transform(Xtest)

reg = linear_model.LinearRegression()
reg.fit(Xtrain, ytrain)
predict = reg.predict(Xtest)
MSE_OLS = metrics.mean_squared_error(ytest,predict)
print(MSE_OLS)

862.4978603032395


In [119]:
import statsmodels.api as sm

mod = sm.OLS(ytrain, Xtrain)
res = mod.fit()

print(res.summary())

                            OLS Regression Results                            
Dep. Variable:         PPI_Likelihood   R-squared:                       0.092
Model:                            OLS   Adj. R-squared:                  0.089
Method:                 Least Squares   F-statistic:                     28.58
Date:                Tue, 08 Jun 2021   Prob (F-statistic):          1.00e-127
Time:                        17:06:53   Log-Likelihood:                -33610.
No. Observations:                7054   AIC:                         6.727e+04
Df Residuals:                    7028   BIC:                         6.745e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                                   coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

In [120]:
#X and Y for market orientation WITH DUMMY
from sklearn.model_selection import train_test_split

X = market_nums.loc[:, market_nums.columns != 'Market_Orientation']
y = market_nums.loc[:, "Market_Orientation"]
Xtrain, Xtest, ytrain, ytest = train_test_split(X_PPI,y_PPI,test_size = .3, random_state = 5)
#1. Fit OLS on the training dataset and compute the test error. ALL with dummy
from sklearn import model_selection, linear_model, neighbors, preprocessing, metrics 
from sklearn.preprocessing import Normalizer
#scaler = preprocessing.StandardScaler()
#print(X_tr)
#Xtrain = scaler.fit_transform(Xtrain)
#Xtest = scaler.transform(Xtest)

reg = linear_model.LinearRegression()
reg.fit(Xtrain, ytrain)
predict = reg.predict(Xtest)
MSE_OLS = metrics.mean_squared_error(ytest,predict)
print(MSE_OLS)

791.8611663461327


In [125]:
import statsmodels.api as sm

mod = sm.OLS(ytrain, Xtrain)
res = mod.fit()

print(res.summary())

                            OLS Regression Results                            
Dep. Variable:         PPI_Likelihood   R-squared:                       0.550
Model:                            OLS   Adj. R-squared:                  0.542
Method:                 Least Squares   F-statistic:                     66.77
Date:                Tue, 08 Jun 2021   Prob (F-statistic):               0.00
Time:                        17:07:41   Log-Likelihood:                -31132.
No. Observations:                7054   AIC:                         6.252e+04
Df Residuals:                    6926   BIC:                         6.340e+04
Df Model:                         127                                         
Covariance Type:            nonrobust                                         
                                                   coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

In [129]:
betas_ppi = reg.coef_
max5_index_ppi = [list(abs(betas_ppi)).index(x) for x in np.sort(abs(betas_ppi))[::-1][:20]]
min5_index_ppi = [list(abs(betas_ppi)).index(x) for x in np.sort(abs(betas_ppi))[:5]]
betas_mo[min5_index_ppi]
Xtrain.columns[min5_index_ppi]
pd.DataFrame([Xtrain.columns[min5_index_ppi], betas_ppi[min5_index_ppi]]).T

,0,1
0,crop_name_1_chickpeas,3.55271e-15
1,crop_name_1_geso,-7.10543e-15
2,crop_name_1_flowers,2.13163e-14
3,crop_name_1_ognons,-5.24025e-14
4,crop_name_1_rambutan,-6.03961e-14
